In [1]:
!pip install mediapipe


In [1]:
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils, optimizers
from PIL import Image
import shutil
from tensorflow.keras import models, layers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.callbacks import EarlyStopping

2023-03-13 10:55:08.140571: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 10:55:08.340621: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-13 10:55:08.346201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dolly/.pyenv/versions/3.10.6/envs/YOGi/lib/python3.10/site-packages/cv2/../..

In [2]:
ground_truths = "../Ground_Truths"

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')
  

Your runtime has 8.2 gigabytes of available RAM

Not using a high-RAM runtime


In [4]:
img_height, img_width = 256, 256
batch_size = 32
poses_list = os.listdir(ground_truths) #Lists all folders in the root directory (raw_data folder)
poses_list

['Shoulder-Pressing_Pose_or_Bhujapidasana_.webp:Zone.Identifier',
 'Scale_Pose_or_Tolasana_.jpeg',
 'Happy_Baby_Pose_or_Ananda_Balasana_.jpeg',
 'Warrior_III_Pose_or_Virabhadrasana_III_.jpeg',
 'Seated_Forward_Bend_pose_or_Paschimottanasana_Seated_Forward_Bend_pose_or_Paschimottanasana_.jpeg:Zone.Identifier',
 'Tree_Pose_or_Vrksasana_.jpeg:Zone.Identifier',
 'Lord_of_the_Dance_Pose_or_Natarajasana_.jpeg',
 'Downward_Facing_Dog_Pose.jpeg:Zone.Identifier',
 'Peacock_Pose.jpeg:Zone.Identifier',
 'Plank_Pose_or_Kumbhakasana_.jpeg',
 'Feathered_Peacock_Pose_or_Pincha_Mayurasana_.jpeg',
 'Extended_Revolved_Triangle_Pose.jpeg:Zone.Identifier',
 'Noose_Pose.jpeg',
 'Extended_Puppy_Pose.jpeg',
 'Bharadvaja_s_Twist_Pose.jpeg:Zone.Identifier',
 'Dolphin_Plank_Pose.jpeg:Zone.Identifier',
 'Eight_Angle_Pose.jpeg',
 'Heron_Pose_or_Krounchasana_.jpeg:Zone.Identifier',
 'Cat_Cow_Pose.jpeg:Zone.Identifier',
 'Scale_Pose_or_Tolasana_.jpeg:Zone.Identifier',
 'Upward_Facing_Two-Foot_Staff_Pose_or_Dwi_Pada

In [15]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks
path = ground_truths # enter dataset path
data = []
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
dataframe = pd.DataFrame(columns = data)

In [20]:
for p in points:
    print(str(p)[13:])

NOSE
LEFT_EYE_INNER
LEFT_EYE
LEFT_EYE_OUTER
RIGHT_EYE_INNER
RIGHT_EYE
RIGHT_EYE_OUTER
LEFT_EAR
RIGHT_EAR
MOUTH_LEFT
MOUTH_RIGHT
LEFT_SHOULDER
RIGHT_SHOULDER
LEFT_ELBOW
RIGHT_ELBOW
LEFT_WRIST
RIGHT_WRIST
LEFT_PINKY
RIGHT_PINKY
LEFT_INDEX
RIGHT_INDEX
LEFT_THUMB
RIGHT_THUMB
LEFT_HIP
RIGHT_HIP
LEFT_KNEE
RIGHT_KNEE
LEFT_ANKLE
RIGHT_ANKLE
LEFT_HEEL
RIGHT_HEEL
LEFT_FOOT_INDEX
RIGHT_FOOT_INDEX


In [17]:
count = 0
for img in os.listdir(path):
    if img[-10:] == "Identifier" or img[-10:] == ".DS_Store" or img[-3:] == "gif":
        pass
    else:
        temp = []
        img = cv2.imread(path + '/' +img)  #cv2.imread(path + "/" + img)
        imageWidth, imageHeight = img.shape[:2]
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        blackie = np.zeros(img.shape) # Blank image
        results = pose.process(imgRGB)
        
    
        if results.pose_landmarks:
            mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
            mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blackie
            landmarks = results.pose_landmarks.landmark
        for i,j in zip(points,landmarks):
            temp = temp + [j.x, j.y, j.z, j.visibility]
            data.loc[count] = temp
            count +=1
            cv2_imshow(img)
            cv2_imshow(blackie)
            cv2.waitKey(100)
#         data.to_csv("ground_truths_keypoints.csv") # save the data as a csv file

AttributeError: 'list' object has no attribute 'loc'

In [11]:
count = 0
for i, j in zip(points,landmarks):
    count += 1

In [18]:
results.pose_landmarks

landmark {
  x: 0.4974498450756073
  y: 0.23743247985839844
  z: -0.32392457127571106
  visibility: 0.9999645948410034
}
landmark {
  x: 0.5081371068954468
  y: 0.2065708041191101
  z: -0.30454471707344055
  visibility: 0.9999871253967285
}
landmark {
  x: 0.5145447254180908
  y: 0.2064111828804016
  z: -0.3046049177646637
  visibility: 0.9999768733978271
}
landmark {
  x: 0.5208553671836853
  y: 0.20658516883850098
  z: -0.30468931794166565
  visibility: 0.9999879598617554
}
landmark {
  x: 0.4853067398071289
  y: 0.20836883783340454
  z: -0.30752259492874146
  visibility: 0.9999887943267822
}
landmark {
  x: 0.4769652485847473
  y: 0.20908987522125244
  z: -0.30758100748062134
  visibility: 0.999980092048645
}
landmark {
  x: 0.4711456298828125
  y: 0.21001595258712769
  z: -0.3076629340648651
  visibility: 0.9999879598617554
}
landmark {
  x: 0.5335362553596497
  y: 0.22053110599517822
  z: -0.18795381486415863
  visibility: 0.9999473094940186
}
landmark {
  x: 0.4634840488433838
  

In [60]:
data

,NOSE_x,NOSE_y,NOSE_z,NOSE_vis,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_vis,LEFT_EYE_x,LEFT_EYE_y,...,RIGHT_HEEL_z,RIGHT_HEEL_vis,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_vis,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_vis


In [56]:
landmarks

[x: 0.5281970500946045
y: 0.2175290733575821
z: -0.05919220671057701
visibility: 0.9686505198478699
, x: 0.5224113464355469
y: 0.21928095817565918
z: -0.04994460567831993
visibility: 0.9717717170715332
, x: 0.5201983451843262
y: 0.21884430944919586
z: -0.05003045126795769
visibility: 0.9748441576957703
, x: 0.5175636410713196
y: 0.21794237196445465
z: -0.05007673427462578
visibility: 0.9743910431861877
, x: 0.5249638557434082
y: 0.22045689821243286
z: -0.08048699796199799
visibility: 0.9758834838867188
, x: 0.5245866775512695
y: 0.22062434256076813
z: -0.08037776499986649
visibility: 0.9764357209205627
, x: 0.5237327814102173
y: 0.22039024531841278
z: -0.0804423913359642
visibility: 0.9743260145187378
, x: 0.49602365493774414
y: 0.21807152032852173
z: -0.007990828715264797
visibility: 0.9722006320953369
, x: 0.5125401020050049
y: 0.2211490273475647
z: -0.13447678089141846
visibility: 0.9733591079711914
, x: 0.520667552947998
y: 0.19115996360778809
z: -0.03926951065659523
visibility: 0.

In [21]:
for p in points:
    print(str(p)[13:])

NOSE
LEFT_EYE_INNER
LEFT_EYE
LEFT_EYE_OUTER
RIGHT_EYE_INNER
RIGHT_EYE
RIGHT_EYE_OUTER
LEFT_EAR
RIGHT_EAR
MOUTH_LEFT
MOUTH_RIGHT
LEFT_SHOULDER
RIGHT_SHOULDER
LEFT_ELBOW
RIGHT_ELBOW
LEFT_WRIST
RIGHT_WRIST
LEFT_PINKY
RIGHT_PINKY
LEFT_INDEX
RIGHT_INDEX
LEFT_THUMB
RIGHT_THUMB
LEFT_HIP
RIGHT_HIP
LEFT_KNEE
RIGHT_KNEE
LEFT_ANKLE
RIGHT_ANKLE
LEFT_HEEL
RIGHT_HEEL
LEFT_FOOT_INDEX
RIGHT_FOOT_INDEX


In [ ]:
vectors_dictionary = {
    "LEFT_ELBOW": [["LEFT_WRIST", "LEFT_SHOULDER"]],
    "RIGHT_ELBOW": [["RIGHT_WRIST", "RIGHT_SHOULDER"]],
    "LEFT_SHOULDER": [["LEFT_ELBOW", "LEFT_HIP"], ["LEFT_HIP", "RIGHT_SHOULDER"], ["LEFT_SHOULDER", "LEFT_KNEE"]],
    "RIGHT_SHOULDER": [["RIGHT_ELBOW", "RIGHT_HIP"], ["RIGHT_HIP", "LEFT_SHOULDER"], ["RIGHT_SHOULDER", "RIGHT_KNEE"]],
    "LEFT_HIP": [["LEFT_SHOULDER", "RIGHT_HIP"], ["RIGHT_HIP", "LEFT_KNEE"]],
    "RIGHT_HIP": [["RIGHT_SHOULDER", "LEFT_HIP"], ["LEFT_HIP", "RIGHT_KNEE"]],
    "LEFT_KNEE": [["LEFT_HIP", "LEFT_ANKLE"]],
    "RIGHT_KNEE": [["RIGHT_HIP", "RIGHT_ANKLE"]]
}